# FASTAPI Tutorial

Fastapi is a python frame work that actually increases the phase of api development, thanks for the asynchronous execution of request using the UVICORN etc.

some of the advantages over other python frameworks are

- **ASGI**, or the Asynchronous Server Gateway Interface, is a Python standard that defines how web servers communicate with asynchronous Python web frameworks and applications. As the successor to the WSGI (Web Server Gateway Interface), ASGI enables non-blocking, real-time communication features like WebSockets and server-sent events, which were not supported by the synchronous WSGI standard. It provides a standardized interface for asynchronous servers and applications, allowing for high-throughput, real-time web applications with better performance and scalability in Python.

- **Typing Hinting** using pydantic library. This increases the productivity of the programmers while coding. Typing Hinting is nothing but showing recommentions on methods and properties based on the data-type

- **Auto Docs**  using swagger UI and redoc. This helps to document the api easily and gives a greate UI for debugging as well!

###  About UVICORN
Steps inside Uvicorn:
1. Create config → builds settings (host, port, app reference, logging).
2. Initialize Server → sets up sockets, protocols, HTTP parsing (via h11 or httptools), and lifespan handlers.
3. Bind to host/port → since you passed 0.0.0.0:8000, it opens a listening socket for connections on all interfaces.
4. Start asyncio tasks → schedules the ASGI app (app) inside the loop, handling HTTP requests asynchronously.
5. Event loop keeps running → but because of nest_asyncio, the notebook doesn’t crash — it keeps processing notebook commands too.

### About Pydantic
All the type casting, data validation, type hinting is all done by pydantic library

### How to run 

There are severals ways to run the fastAPI app.
1. Using UVICORN
    - uvicorn \<filename\>:\<fastapi Instance\> --reload -> on cmd
    - uvicorn.run(\<fastapi Instance\>, host="0.0.0.0", port=8000) -> on code
2. Using FastAPI CMD
    - fastapi dev \<filename\> -> on cmd to run on development mode
    - fastapi run \<filename\> -> on cmd to run on production mode

## Basics

In [174]:
from fastapi import FastAPI

In [175]:
app = FastAPI() # Here the app is the instance of the FASTAPI

In [176]:
@app.get("/") #  Here @ is a decorator used to  decorate the function below 
def base_route():
    return "Welcome to the base page!"

# Here the get is the operation, '/' is the path or route or endpoint, and the function is called path operation function

#### Dynamic Route or Path Parameters
**Note**: The dynamic route must always present after the static routes. This ensures that the dynamic route won't recognize the static route and process the unexpected data. FastAPI always matches the path from top to bottom so placing the dynamic route after the static route will result in the expected outcome

In [177]:
@app.get("/path-param{id}")
def dynamic_param(id: int): # If the id is not integer fastapi will automatically throw an error a as response
    return {"data": f"data of blog {id}"}

#### Query and optional Parameter

In [178]:
@app.get("/query-param")
def query_param(limit: int): # Here limit is a query parameter which is specified in the URL using '?' and '&'
    return {"Query Param":limit}

@app.get("/optional-param")
def optional_param(param: str | None=None):  # Here '|' is used to mention that it is optional 
    return {"Optional Param": param}


#### Request Body

In [179]:
from pydantic import BaseModel

class RequestBody(BaseModel):
    param1: int
    param2: str
    param3: bool

@app.post("/request-body")
def request_body(request: RequestBody):
    return {"Data": request}


#### Creating and linking SQL model to endpoints using SQLAlchemy

In [180]:
from sqlalchemy import create_engine, Column, String, Integer, Float
from sqlalchemy.orm import declarative_base, sessionmaker

In [181]:
DATABASE_URL = "sqlite:///fastApiDatabase.db"
engine = create_engine(DATABASE_URL, echo=True)
Base = declarative_base()

In [182]:
class Blog(Base):
    __tablename__ = "blog"
    id: int = Column(Integer, primary_key=True)
    title: str = Column(String, nullable=False)
    description: str = Column(String, nullable=False)
    price: float = Column(Float, nullable=False)
    author: str = Column(String, nullable=False)

Base.metadata.create_all(bind=engine)

2025-09-21 23:50:56,129 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 23:50:56,130 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("blog")
2025-09-21 23:50:56,130 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-21 23:50:56,131 INFO sqlalchemy.engine.Engine COMMIT


In [183]:
from fastapi import Depends
from sqlalchemy.orm import Session

def get_db():
    Session = sessionmaker(engine)
    db = Session()
    try:
        yield db
    finally:
        db.close()

@app.get("/blogs/all")
def get_all_blogs(db: Session = Depends(get_db)):
    return db.query(Blog).all()

#### CRUD Operations using SQLAlchemy ORM

In [184]:
from pydantic import BaseModel

class BlogSchema(BaseModel):
    title: str
    description: str
    price: float
    author: str

@app.get("/blogs/{id}")
def get_blog_by_id(id: int, db: Session = Depends(get_db)):
    return db.query(Blog).filter(Blog.id == id).first()

@app.post("/blog/create")
def create_blog(blog: BlogSchema, db: Session = Depends(get_db)):
    blog = Blog(title=blog.title, description=blog.description, price=blog.price, author=blog.author)
    db.add(blog)
    db.commit()
    db.refresh(blog)
    return blog

@app.put("/blog/update/{id}")
def update_blog(id:int, blog: BlogSchema, db: Session = Depends(get_db)):
    db.query(Blog).filter(Blog.id == id).update({
        "title":blog.title, 
        "description":blog.description, 
        "price":blog.price, 
        "author":blog.author
    })
    db.commit()
    return blog

@app.delete("/blog/delete/{id}")
def delete_blog(id: int, db: Session = Depends(get_db)):
    try:
        db.query(Blog).filter(Blog.id == id).delete()
        db.commit()
        return {"status": f"Successfully deleted blog {id}"}
    except():
        return {"status": f"Deletion of blog {id} Failed!"}

#### Exception and Response Code

In [185]:
from fastapi import status, HTTPException, Response

# Using the decorator
@app.get("/exception", status_code=status.HTTP_401_UNAUTHORIZED) 
def exception():
    return {"data": "Exception raised!"}

# Using the HTTPException
@app.get("/manual-exception", status_code=status.HTTP_401_UNAUTHORIZED)
def manual_exception():
    try:
        raise Exception()
    except:
        raise HTTPException(status_code=status.HTTP_418_IM_A_TEAPOT, detail="Manual Exception Raised!")
    return {"data": "This will not be executed"}

# Using the Response Class
@app.get("/added-exception")
def added_exception(response: Response):
    try:
        raise Exception()
    except:
        response.status_code = status.HTTP_418_IM_A_TEAPOT
    return {"data":"nothing"}

#### Response Model

In [186]:
from pydantic import BaseModel
from typing import List

class ResponseModel(BaseModel):
    title: str
    author: str

    class Config:
        from_attributes = True

@app.get("/blog/response-model/all", response_model=List[ResponseModel])
def get_all_blogs_using_response_model(db: Session = Depends(get_db)):
    blogs = db.query(Blog).all()
    return blogs

@app.get("/blog/response-model/{id}", response_model=ResponseModel)
def get_blog_using_response_model(id: int, db: Session = Depends(get_db)):
    blog = db.query(Blog).filter(Blog.id == id).first()
    return blog

#### Running FastAPI inside Jupyter Notebook

To run fastapi inside jupyter notebook we need to use nest_asyncio.
- uvicorn.run() internally calls asyncio.run().
- But Jupyter/IPython already runs an event loop for you.
- Python forbids nesting asyncio.run() inside an already running loop, so you get:
    - **RuntimeError: asyncio.run() cannot be called from a running event loop**

1. Jupyter already has an event loop
- Jupyter (IPython kernel) runs an asyncio event loop in the background to manage code execution, message passing, widgets, etc.
Normally, calling asyncio.run() inside an already running loop raises:
RuntimeError: asyncio.run() cannot be called from a running event loop

2. nest_asyncio.apply()
-  This function patches the current event loop so that it can be nested.
Normally, asyncio forbids re-entering the same loop (asyncio.run always expects a fresh loop).
After patching, the existing Jupyter event loop can re-enter itself — allowing libraries like Uvicorn, aiohttp, etc. to run without crashing.
Think of it as: “let me allow recursion on the event loop.”

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [7084]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53237 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53237 - "GET /openapi.json HTTP/1.1" 200 OK
2025-09-21 23:51:09,479 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 23:51:09,491 INFO sqlalchemy.engine.Engine SELECT blog.id AS blog_id, blog.title AS blog_title, blog.description AS blog_description, blog.price AS blog_price, blog.author AS blog_author 
FROM blog
2025-09-21 23:51:09,494 INFO sqlalchemy.engine.Engine [generated in 0.00268s] ()
2025-09-21 23:51:09,501 INFO sqlalchemy.engine.Engine ROLLBACK
INFO:     127.0.0.1:51215 - "GET /blog/response-model/all HTTP/1.1" 200 OK
2025-09-21 23:51:21,326 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 23:51:21,329 INFO sqlalchemy.engine.Engine SELECT blog.id AS blog_id, blog.title AS blog_title, blog.description AS blog_description, blog.price AS blog_price, blog.author AS blog_author 
FROM blog 
WHERE blog.id = ?
 LIMIT ? OFFSET ?
2025-09-21 23:51:21,329 INFO sqlalchemy.engine.Engine [generated